# Model Abstraction

Our model for Machine Learning for Complex Pricing Models is mainly based on LightGBM, due to its computation speed and effectiveness. The reason for why our model could generate high accuracy is that not only we disorganized the raw data and split them in to mini batches to avoid overfitting and memory explosion, but also we found the optimal value of hyper parameters as accurate as possible. Apart from it, this model has great explain ability as well because of low memory cost, straightforward logic and less parameters. More details about model building process will be explained in the introduction part.

-----------

# Model Introduction

###  Step 1:   Mini batch generation and Raw data disorganization

Mini Batch generation:
1. Due to RAM memory limitation, it's not possible to train all the datasets at the same  time.
2. To avoid memory explosion, we split the whole datasets into 345 mini batches.
    
Raw Data Disorganization:
1. According to our exploratory data analysis, we found that the distribution of each raw dataset (such as "all_data_df(0,1,2).joblib") is different.
2. To avoid overfitting on any raw dataset, we disorganized raw data and combine them into mini batches randomly.

### Step 2: Data standardization

1. To standardize data, we fitted our scaler with a certain number of mini batches (124_final_model_scaler.pickle).
2. We utilized the fitted scaler to transform all the data.

### Step 3: Feature creation

1. Based on the transformation on single and multiple features, we created 432 new features from original 164 features.
2. More details are given in the function for data engineering.

### Step 4: Feature Selection

1. As we would choose LightGBM as our final model, the final features are chosen by the feature importance given by LightGBM.
2. Eventually, 123 well-performing features are  selected, which contribute over 90% importance in total.

### Step 5: Model Selection

According to prediction accuracy and computation speed, LightGBM is decided as our final model among all the machine learning models.

### Step 6: Fine-tune parameters

Through numerous tests, we fine-tuned the parameters for LightGBM model, including max_depth, num_leaves and etc.

### Step 7: Model training

1. As we have 340 mini batches as training data and take 5 mini batches in each epoch, there will be 68 epoches for model training in total.
2. After 68 epoches training, the final model is generated for further prediction.

### Step 8: Prediction 

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# The reason why this model has great explain ability 

1. Our model is with few hyper parameters (8) among a lots of those could be chosen in LightGBM. 
2. The logic of our model is quiet straightforward based on LightGBM as what described in the introduction.
3. The RAM memory used for model training is quite few, just around 1G out of 4G.
4. This model has smooth sensitivities on the inputs and great generalization ablity.

------

# Function for data engineering 

### Data Engineering / Feature Transformation

In this version, we first generate 432 features from originally given 164 features and then implement feature selection based on feature importance derived from tree model to get 123 well-performing features which contribute over 90% importances overall. Names of them can be found in 'final_model_feature.pkl'. Before fed into our final model, these features will be standardized using parameters from 'final_model_scaler.pkl' that is fitted with our entire training set.

The originally given 164 features can be divided into two groups. The first group of features is about underlying asset, the other group is about contract. The first group features can be further divided into two parts, the first part is about vol, skew and curve; the other part is about vov, svr and mr that show numeric regularity in the dataset. We do four levels of generation.

(1) We first consider features generated from one type of variables under single underlying asset. For example, we take mean, max, min on variable 'vovatm' across all time window under a single underlying asset like Undl0:

* Undl0volatm_mean = mean(Undl0vovatm_6M,Undl0vovatm_1Y,...,Undl0vovatm_5Y,Undl0vovatm_6Y)

* Undl0volatm_max = max(Undl0vovatm_6M,Undl0vovatm_1Y,...,Undl0vovatm_5Y,Undl0vovatm_6Y)

* Undl0volatm_min = min(Undl0vovatm_6M,Undl0vovatm_1Y,...,Undl0vovatm_5Y,Undl0vovatm_6Y)

We do this for volatm, skew95.105, skew70.130, curve95.105, curve70.130 and for all three underlyings(because we don't know which asset has any priority over other two so they should be treated equally in this process) to **capture fluctuation features on time-series level for each underlying asset**, which is implemented in function 'FeatureGen_SingleAsset_OneVar' inside our 'data_engineering' function.

(2) Then we consider features from different type of variables under single underlying asset. For example, we multiply 'volatm' by 'skew' that are of same window under single underlying asset like Undl0:

* Undl0centMom3.95.105_6M = Undl0volatm_6M * Undl0skew95.105_6M

* Undl0centMom3.70.130_1Y = Undl0volatm_1Y * Undl0skew70.130_1Y

* ...

We do this for all three underlyings. Since we're lacked of information of what variable 'curve' means, we only do this multiplication with volatm and skew. **This will generate the third central moment(at least something similar to)**. We also take mean, max, min for those 'centMom3.xxxx' like we did in (1). The implementation can be found in function 'FeatureGen_SingleAsset_TwoVar' inside our 'data_engineering' function.

(3) Furthermore, we consider features from same type of variables across all three underlyings. For example, we take mean, max and min for 'volatm_6M' across Undl0, Undl1, Undl2:

* All_volatm_6M_mean = mean(Undl0volatm_6M,Undl1volatm_6M,Undl2volatm_6M)

* All_volatm_1Y_max = max(Undl0volatm_1Y,Undl1volatm_1Y,Undl2volatm_1Y)

* All_volatm_1Y6M_min = min(Undl0volatm_1Y6M,Undl1volatm_1Y6M,Undl2volatm_1Y6M)

* ...

We do this for all volatm, skew, curve. **This is reasonalbe since the price of the autocall option is strongly related with the best/worst performed underlying on the observation day**. We implemented this in function 'FeatureGen_MultiAsset' inside our 'data_engineering' function.

(4) We also squared originally given 159 numeric features to have a better numeric magnitude for our algo to iterate and learn. This is implemented in function 'FeatureGen_Square' inside our 'data_engineering' function.

In [1]:
def data_engineering(df, pickle_path):
    """
    df --- X values from test data
    pickle_path --- the path where stores "124_final_model_features.pickle" and "124_final_model_scaler.pickle", e.g. "/home/jguo81/FordhamA/"
    """
   
    import re
    import numpy as np
    import pandas as pd
   
    def FeatureGen_SingleAsset_OneVar(df, Asset):
        """
        generate features mentioned in (1)
        --------
        :param Asset: 'Undl0', 'Undl1', 'Undl2'
        :param df: original df of 50 Xs for Asset
        """
        import pandas as pd

        # new dataframe to store generated features
        newdf=pd.DataFrame()

        # columns of vol
        Vols=[Asset+'volatm_6M',Asset+'volatm_1Y',Asset+'volatm_1Y6M',Asset+'volatm_2Y',Asset+'volatm_2Y6M',
              Asset+'volatm_3Y',Asset+'volatm_3Y6M',Asset+'volatm_4Y',Asset+'volatm_5Y',Asset+'volatm_6Y']

        # columns of skew95.105
        Skews95_105=[Asset+'skew95.105_6M',Asset+'skew95.105_1Y',Asset+'skew95.105_1Y6M',
                     Asset+'skew95.105_2Y',Asset+'skew95.105_2Y6M',Asset+'skew95.105_3Y',
                     Asset+'skew95.105_3Y6M',Asset+'skew95.105_4Y',Asset+'skew95.105_5Y',
                     Asset+'skew95.105_6Y']

        # columns of skew70.130
        Skews70_130=[Asset+'skew70.130_6M',Asset+'skew70.130_1Y',Asset+'skew70.130_1Y6M',
                     Asset+'skew70.130_2Y',Asset+'skew70.130_2Y6M',Asset+'skew70.130_3Y',
                     Asset+'skew70.130_3Y6M',Asset+'skew70.130_4Y',Asset+'skew70.130_5Y',
                     Asset+'skew70.130_6Y']

        # columns of curve95.105
        Curves95_105=[Asset+'curve95.105_6M',Asset+'curve95.105_1Y',Asset+'curve95.105_1Y6M',
                     Asset+'curve95.105_2Y',Asset+'curve95.105_2Y6M',Asset+'curve95.105_3Y',
                     Asset+'curve95.105_3Y6M',Asset+'curve95.105_4Y',Asset+'curve95.105_5Y',
                     Asset+'curve95.105_6Y']

        # columns of skew70.130
        Curves70_130=[Asset+'curve70.130_6M',Asset+'curve70.130_1Y',Asset+'curve70.130_1Y6M',
                     Asset+'curve70.130_2Y',Asset+'curve70.130_2Y6M',Asset+'curve70.130_3Y',
                     Asset+'curve70.130_3Y6M',Asset+'curve70.130_4Y',Asset+'curve70.130_5Y',
                     Asset+'curve70.130_6Y']

        # mean, max, min of vols across different windows of a single underyling
        newdf[Asset+'volatm_mean']=df[Vols].mean(axis=1)
        newdf[Asset+'volatm_max']=df[Vols].max(axis=1)
        newdf[Asset+'volatm_min']=df[Vols].min(axis=1)

        # mean, max, min of skew95.105 across different windows of a single underyling
        newdf[Asset+'skew95.105_mean']=df[Skews95_105].mean(axis=1)
        newdf[Asset+'skew95.105_max']=df[Skews95_105].max(axis=1)
        newdf[Asset+'skew95.105_min']=df[Skews95_105].min(axis=1)

        # mean, max, min of skew70.130 across different windows of a single underyling
        newdf[Asset+'skew70.130_mean']=df[Skews70_130].mean(axis=1)
        newdf[Asset+'skew70.130_max']=df[Skews70_130].max(axis=1)
        newdf[Asset+'skew70.130_min']=df[Skews70_130].min(axis=1)

        # mean, max, min of curve95.105 across different windows of a single underyling
        newdf[Asset+'curve95.105_mean']=df[Curves95_105].mean(axis=1)
        newdf[Asset+'curve95.105_max']=df[Curves95_105].max(axis=1)
        newdf[Asset+'curve95.105_min']=df[Curves95_105].min(axis=1)

        # mean, max, min of curve70.130 across different windows of a single underyling
        newdf[Asset+'curve70.130_mean']=df[Curves70_130].mean(axis=1)
        newdf[Asset+'curve70.130_max']=df[Curves70_130].max(axis=1)
        newdf[Asset+'curve70.130_min']=df[Curves70_130].min(axis=1)

        return newdf
   
    def FeatureGen_SingleAsset_TwoVar(df, Asset):
        """
        genearate features mentioned in (2)
        :param Asset: 'Undl0', 'Undl1', 'Undl2'
        :param df: original df of 50 Xs for Asset
        """
        import pandas as pd

        # new dataframe to store generated features
        newdf=pd.DataFrame()

        # multiply vol by skew95.105 with same window to get the third central momentum of a single underlying
        newdf[Asset+'centMom3.95.105_6M']=df[Asset+'volatm_6M']*df[Asset+'skew95.105_6M']
        newdf[Asset+'centMom3.95.105_1Y']=df[Asset+'volatm_1Y']*df[Asset+'skew95.105_1Y']
        newdf[Asset+'centMom3.95.105_1Y6M']=df[Asset+'volatm_1Y6M']*df[Asset+'skew95.105_1Y6M']
        newdf[Asset+'centMom3.95.105_2Y']=df[Asset+'volatm_2Y']*df[Asset+'skew95.105_2Y']
        newdf[Asset+'centMom3.95.105_2Y6M']=df[Asset+'volatm_2Y6M']*df[Asset+'skew95.105_2Y6M']
        newdf[Asset+'centMom3.95.105_3Y']=df[Asset+'volatm_3Y']*df[Asset+'skew95.105_3Y']
        newdf[Asset+'centMom3.95.105_3Y6M']=df[Asset+'volatm_3Y6M']*df[Asset+'skew95.105_3Y6M']
        newdf[Asset+'centMom3.95.105_4Y']=df[Asset+'volatm_4Y']*df[Asset+'skew95.105_4Y']
        newdf[Asset+'centMom3.95.105_5Y']=df[Asset+'volatm_5Y']*df[Asset+'skew95.105_5Y']
        newdf[Asset+'centMom3.95.105_6Y']=df[Asset+'volatm_6Y']*df[Asset+'skew95.105_6Y']

        # multiply vol by skew70.130 with same window to get the third central momentum of a single underlying
        newdf[Asset+'centMom3.70.130_6M']=df[Asset+'volatm_6M']*df[Asset+'skew70.130_6M']
        newdf[Asset+'centMom3.70.130_1Y']=df[Asset+'volatm_1Y']*df[Asset+'skew70.130_1Y']
        newdf[Asset+'centMom3.70.130_1Y6M']=df[Asset+'volatm_1Y6M']*df[Asset+'skew70.130_1Y6M']
        newdf[Asset+'centMom3.70.130_2Y']=df[Asset+'volatm_2Y']*df[Asset+'skew70.130_2Y']
        newdf[Asset+'centMom3.70.130_2Y6M']=df[Asset+'volatm_2Y6M']*df[Asset+'skew70.130_2Y6M']
        newdf[Asset+'centMom3.70.130_3Y']=df[Asset+'volatm_3Y']*df[Asset+'skew70.130_3Y']
        newdf[Asset+'centMom3.70.130_3Y6M']=df[Asset+'volatm_3Y6M']*df[Asset+'skew70.130_3Y6M']
        newdf[Asset+'centMom3.70.130_4Y']=df[Asset+'volatm_4Y']*df[Asset+'skew70.130_4Y']
        newdf[Asset+'centMom3.70.130_5Y']=df[Asset+'volatm_5Y']*df[Asset+'skew70.130_5Y']
        newdf[Asset+'centMom3.70.130_6Y']=df[Asset+'volatm_6Y']*df[Asset+'skew70.130_6Y']

        # mean, max, min of generated third central momentum across different window
        newdf[Asset+'centMom3.95.105_mean']=newdf.iloc[:,:10].mean(axis=1)
        newdf[Asset+'centMom3.95.105_max']=newdf.iloc[:,:10].max(axis=1)
        newdf[Asset+'centMom3.95.105_min']=newdf.iloc[:,:10].min(axis=1)

        # mean, max, min of generated third central momentum across different window
        newdf[Asset+'centMom3.70.130_mean']=newdf.iloc[:,10:20].mean(axis=1)
        newdf[Asset+'centMom3.70.130_max']=newdf.iloc[:,10:20].max(axis=1)
        newdf[Asset+'centMom3.70.130_min']=newdf.iloc[:,10:20].min(axis=1)

        return newdf
   
    def FeatureGen_MultiAssets(df):
        """
        generate features mentioned in (3)
        --------
        :param df: df of 150 Xs
        """

        import pandas as pd

        newdf=pd.DataFrame()
        cols=df.columns[:50]

        for i in range(50):
            # mean, max, min of each feature (volatm_xx, skew_xxxx, curve_xxxx) across all three underlyings
            newdf['All_'+cols[i][5:]+'_mean']=df.iloc[:,[i,i+50,i+100]].mean(axis=1)
            newdf['All_'+cols[i][5:]+'_max']=df.iloc[:,[i,i+50,i+100]].max(axis=1)
            newdf['All_'+cols[i][5:]+'_min']=df.iloc[:,[i,i+50,i+100]].min(axis=1)

        return newdf
   
    def FeatureGen_Square(df):
        """
        generate features mentioned in (4)
        ---------
        :param df: df of orginal 159 Xs
        """
        
        # squared features
        newdf=df**2
        newdf.columns=list(map(lambda x: x+'_square',df.columns))

        return newdf
   
    def FeatureGeneration(df):
        """
        generate all those 4 classes features
        -----------
        :param df: df of a mini-batch dataset
        """
        import pandas as pd

        Xs=df.iloc[:,:150]
        Assets=['Undl0','Undl1','Undl2']

        # generate features in (1) and (2)
        for i, asset in enumerate(Assets):
            newdf1=FeatureGen_SingleAsset_OneVar(Xs.iloc[:,(i*50):((i+1)*50)],asset)
            newdf2=FeatureGen_SingleAsset_TwoVar(Xs.iloc[:,(i*50):((i+1)*50)],asset)
            Xs=pd.concat([Xs,newdf1,newdf2], axis=1)

        # generate features in (3)
        newdf=FeatureGen_MultiAssets(Xs.iloc[:,:150])
        Xs=pd.concat([Xs,newdf], axis=1)

        # generate features in (4)
        newdf=FeatureGen_Square(df.iloc[:,:159])
        Xs=pd.concat([Xs,newdf,df.iloc[:,-14:]],axis=1)

        return Xs
     
    # change dtypes to lower but appropriate precision to reduce memory usage
    df=df.astype(np.float16)
    df['ContractFeature_Schedule,EndDate_ENCODED']=df['ContractFeature_Schedule,EndDate_ENCODED']\
                                                        .astype(np.int).astype(str).astype('category')
   
    
    # Use FeatureGeneration to generate 432 new features
    df=FeatureGeneration(df)
    
    # Delete the punctuation marks in the names of features to avoid error, when importing them in LightGBM training
    df2 = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
   
    # import 123 selected features that are chosen by the feature importance via LightGBM model  
    with open(pickle_path+'124_final_model_features.pickle', 'rb') as file:
        features = pickle.load(file)
   
    # Delete label (y value) from feature list
    features.pop(-1)
    
    # Select 123 features among 596 features (164+432)
    df3 = df2.loc[:,features]
   
    # import fitted scaler for standardization
    with open(pickle_path+'124_final_model_scaler.pickle', 'rb') as file:
        scaler = pickle.load(file)
       
    # Standardiza by the scaler
    col = df3.columns
    index = df3.index
    df4 = scaler.transform(df3)
    df5 = pd.DataFrame(df4, columns = col, index = index)
   
    # Transform each feature's type into catefory to save memory
    for j in df5:
        df5[j] = df5[j].astype("category")
   
   
    return df5

# Function for model training 

1. As we have 340 mini batches as training data and take 5 mini batches in each epoch, there will be 68 epoches for model training in total.
2. After 68 epoches training, the final model is generated for further prediction.

In [8]:
def model_training(data_path, pickle_path):
    
    """
    data_path --- the path where stores the needed data for model training, e.g.'/home/jguo81/FordhamA/JiaGuo/Data/124_final_model/'
    pickle_path --- the path where stores "124_final_model_scaler.pickle", e.g. "/home/jguo81/FordhamA/"
    """
    
    import os
    from joblib import load
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    import lightgbm as lgb
    import gc
    from sklearn import metrics

    # Open the path where stores the mini batched of data for model training
    os.chdir(data_path)
    
    # Get a list of names of mini batches of data
    files=os.listdir()
     
    # There are 345 mini batches in total. And we use 340 mini batches as training data, 2 batches of data as validation data and 3 batches of data as test data respectively
    train_file = files[:-5]
    eval_file = files[-5:-3]
    test_file = files[-3:]
    
    # Import the scaler which is used to standardize our training data
    with open(pickle_path+'124_final_model_scaler.pickle', 'rb') as file:
        scaler = pickle.load(file)
    
    
    # Function used to standardize a certain number of mini batches and combine them as a training batch. In the further training part, we use a training batch in an epoch.
    def scale_combine(file_list, data_path):
        """
        file_list --- a list that contain a certain number of mini batches, which will be combined as a training batch for a training epoch
        data_path --- the path where stores the needed data for model training, e.g."/home/jguo81/FordhamA/JiaGuo/Data/124_final_model/"
        """
        
        # Load the first mini batch in the file_list
        train_data = load(data_path+file_list[0])
        
        # Split features and label as X and y
        X_train, y_train = train_data.iloc[:,:-1], train_data.iloc[:,-1]
        col = X_train.columns
        index = X_train.index
        
        # Standardize the X values of the first mini batch
        X_train = scaler.transform(X_train)
        
        # Transform standardized X values as dataframe
        X_train = pd.DataFrame(X_train, columns = col, index = index)
        
        # For the rest of mini batches in the file list, we conduct the same procedure as the first mini batch and combine them together.
        for i in file_list[1:]:
            new = load(path+i)
            X_new, y_new = new.iloc[:,:-1], new.iloc[:,-1]
            X_new = scaler.transform(X_new)
            X_new = pd.DataFrame(X_new, columns = col, index = index)
            X_train = pd.concat([X_train,X_new])
            y_train = pd.concat([y_train,y_new])

        return X_train, y_train
    
    # Standardize and combine the mini batches for validation and test respectively
    X_eval, y_eval = scale_combine(eval_file, path)
    X_test, y_test = scale_combine(test_file, path)
    
    # Delete the punctuation marks in the names of features to avoid error, when importing them in LightGBM training
    X_eval_df = X_eval.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_test_df = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    # Transform each feature's type into catefory to save memory
    for i in X_eval_df:
        X_eval_df[i] = X_eval_df[i].astype("category")
    for i in X_test_df:
        X_test_df[i] = X_test_df[i].astype("category")
    
    # Initiated LightGBM model with fine-tuned parameters
    lgb_model = lgb.LGBMRegressor(objective = 'regression',
                     is_unbalance = True,
                     metric = 'rmse',
                     max_depth = 15,
                     num_leaves = 120,
                     learning_rate = 0.1,
                     feature_fraction = 1,
                     min_child_samples=30,
                     min_child_weight=0.001,
                     bagging_fraction = 1,
                     bagging_freq = 2,
                     reg_alpha = 0.001,
                     reg_lambda = 8,
                     cat_smooth = 0,
                     num_iterations = 400)
    
    mae = []
    mse = []
    
    # Because there are 340 mini batches for model training and we use 5 mini batches in on epoch, there will be 51 epoches in model training
    for i in range(68):

        # Get 5 mini batches from train file for each epoch
        train_file_loop = train_file[i*5:i*5+5]

        # print the file names used in this epoch
        for m in range(5):
            print("Train file used: ", i*5+m,"----------",train_file_loop[m])
        print("----------------------------")

        # Standardize and combine the mini batches for each epoch
        X_train_loop, y_train_loop = scale_combine(train_file_loop, path)
        
        # Delete the punctuation marks in the names of features to avoid error when importing them in LightGBM training
        X_train_df_loop = X_train_loop.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

        # Transform each feature's type into catefory to save memory
        for j in X_train_df_loop:
            X_train_df_loop[j] = X_train_df_loop[j].astype("category")

        # Train our lightGBM model for each epoch. After one epoch, this LightGBM model will be trained continuously in next epoch untill 68 epoches end.
        lgb_model.fit(X_train_df_loop, y_train_loop, 
        eval_metric='l1',
        eval_set=[(X_eval_df, y_eval)],
        verbose = True,
        early_stopping_rounds=200)

        # Use garbage collector to purge the garbage in the buffer area
        gc.collect()

        print("-----------------------------------------------------------------------------------")
        print("Round: ",i)

        # Get predicted y with test datasets in each epoch to be aware of the effectiveness of our model in each epoch
        y_pred = lgb_model.predict(X_test_df)
        ae = metrics.max_error(y_test, y_pred)
        se = metrics.mean_squared_error(y_test, y_pred)
        print("MAE:",ae)
        print("MSE:",se)

        mae.append(ae)
        mse.append(se)
        print("-----------------------------------------------------------------------------------")
        
    
    return lgb_model

Model training

In [19]:
lgb_model = model_training("/home/jguo81/FordhamA/JiaGuo/Data/124_final_model/", "/home/jguo81/FordhamA/")

Train file used:  0 ---------- 124_microBatch_290.joblib
Train file used:  1 ---------- 124_microBatch_129.joblib
Train file used:  2 ---------- 124_microBatch_288.joblib
Train file used:  3 ---------- 124_microBatch_314.joblib
Train file used:  4 ---------- 124_microBatch_24.joblib
----------------------------


/mnt/public/IT/Libs/Python/external/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


/mnt/public/IT/Libs/Python/external/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/mnt/public/IT/Libs/Python/external/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's l1: 0.00741848
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.00691906
[3]	valid_0's l1: 0.00648829
[4]	valid_0's l1: 0.00609712
[5]	valid_0's l1: 0.00575322
[6]	valid_0's l1: 0.00544022
[7]	valid_0's l1: 0.00516851
[8]	valid_0's l1: 0.00492489
[9]	valid_0's l1: 0.00471207
[10]	valid_0's l1: 0.00451716
[11]	valid_0's l1: 0.004334
[12]	valid_0's l1: 0.00418121
[13]	valid_0's l1: 0.00403951
[14]	valid_0's l1: 0.00390836
[15]	valid_0's l1: 0.00377553
[16]	valid_0's l1: 0.00366521
[17]	valid_0's l1: 0.00354996
[18]	valid_0's l1: 0.00345797
[19]	valid_0's l1: 0.00336769
[20]	valid_0's l1: 0.00328448
[21]	valid_0's l1: 0.00321072
[22]	valid_0's l1: 0.00314194
[23]	valid_0's l1: 0.00308017
[24]	valid_0's l1: 0.0030271
[25]	valid_0's l1: 0.00297132
[26]	valid_0's l1: 0.00289901
[27]	valid_0's l1: 0.00284391
[28]	valid_0's l1: 0.00278005
[29]	valid_0's l1: 0.00274035
[30]	valid_0's l1: 0.00269002
[31]	valid_0's l1: 0.00264582
[32]	valid_0's l1: 0

...

[1]	valid_0's l1: 0.00742313
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.00691453
[3]	valid_0's l1: 0.00647799
[4]	valid_0's l1: 0.00607656
[5]	valid_0's l1: 0.0057265
[6]	valid_0's l1: 0.00541865
[7]	valid_0's l1: 0.00514223
[8]	valid_0's l1: 0.00489752
[9]	valid_0's l1: 0.00467789
[10]	valid_0's l1: 0.00448869
[11]	valid_0's l1: 0.00431741
[12]	valid_0's l1: 0.00416271
[13]	valid_0's l1: 0.00402095
[14]	valid_0's l1: 0.00389505
[15]	valid_0's l1: 0.0037678
[16]	valid_0's l1: 0.00365662
[17]	valid_0's l1: 0.00356547
[18]	valid_0's l1: 0.00346835
[19]	valid_0's l1: 0.00338264
[20]	valid_0's l1: 0.00329781
[21]	valid_0's l1: 0.0032283
[22]	valid_0's l1: 0.00316384
[23]	valid_0's l1: 0.00309958
[24]	valid_0's l1: 0.00303726
[25]	valid_0's l1: 0.00296762
[26]	valid_0's l1: 0.00291174
[27]	valid_0's l1: 0.00285612
[28]	valid_0's l1: 0.00280212
[29]	valid_0's l1: 0.00275873
[30]	valid_0's l1: 0.00272615
[31]	valid_0's l1: 0.00267368
[32]	valid_0's l1: 0

/mnt/public/IT/Libs/Python/external/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


/mnt/public/IT/Libs/Python/external/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/mnt/public/IT/Libs/Python/external/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's l1: 0.00740237
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.00689983
[3]	valid_0's l1: 0.00645612
[4]	valid_0's l1: 0.00606024
[5]	valid_0's l1: 0.00570932
[6]	valid_0's l1: 0.00538683
[7]	valid_0's l1: 0.0051155
[8]	valid_0's l1: 0.00487579
[9]	valid_0's l1: 0.00464774
[10]	valid_0's l1: 0.00445352
[11]	valid_0's l1: 0.00428145
[12]	valid_0's l1: 0.00412847
[13]	valid_0's l1: 0.0039777
[14]	valid_0's l1: 0.00385125
[15]	valid_0's l1: 0.00373375
[16]	valid_0's l1: 0.0036225
[17]	valid_0's l1: 0.00352776
[18]	valid_0's l1: 0.00342885
[19]	valid_0's l1: 0.00334031
[20]	valid_0's l1: 0.00326683
[21]	valid_0's l1: 0.00319844
[22]	valid_0's l1: 0.00312673
[23]	valid_0's l1: 0.00306161
[24]	valid_0's l1: 0.00299118
[25]	valid_0's l1: 0.00293549
[26]	valid_0's l1: 0.00288412
[27]	valid_0's l1: 0.00282499
[28]	valid_0's l1: 0.00277027
[29]	valid_0's l1: 0.00273142
[30]	valid_0's l1: 0.0026818
[31]	valid_0's l1: 0.00264468
[32]	valid_0's l1: 0.

Store the pretrained model in a pickle file

In [86]:
import pickle

with open('/home/jguo81/FordhamA/model/124_final_model.pickle', 'wb') as file:
    pickle.dump(lgb_model, file)

Load the pretrained model from a pickle file

In [4]:
with open('/home/jguo81/FordhamA/124_final_model.pickle', 'rb') as file:
    lgb_model = pickle.load(file)

# Function for Prediction 

In [2]:
def prediction(model, data):
    
    """
    model --- the pretrained lightgbm model
    data --- the data processed by data_engineering fuction (X value)
    """
    
    import lightgbm as lgb
    
    y_pred = model.predict(data)
    
    return y_pred

# Running example

Acquire the test data example, and we use "all_data_df(2, 4, 8).joblib" without labels (X value) here

In [12]:
import os
from joblib import load
import pickle

os.chdir("/home/jguo81/UBSPRICING/Data/")
test_data = load("all_data_df(2, 4, 8).joblib")
X_test = test_data.iloc[:,:164]

Data engineering

You should change pickle_path to your own path where stores the pickle files.

In [13]:
pickle_path = "/home/jguo81/FordhamA/"
X_test = data_engineering(X_test, pickle_path)

Load the pretrained model from a pickle file

In [14]:
with open('/home/jguo81/FordhamA/124_final_model.pickle', 'rb') as file:
    lgb_model = pickle.load(file)

Make predictions

In [15]:
y_pred = prediction(lgb_model, X_test)

Use labels in "all_data_df(2, 4, 8).joblib" to test our model's performance

In [16]:
y_test = test_data.iloc[:,-1]

In [17]:
from sklearn import metrics

print("MAE:",metrics.max_error(y_test, y_pred))
print("MSE:",metrics.mean_squared_error(y_test, y_pred))

MAE: 0.03973101142391629
MSE: 1.4873366853155038e-05
